In [1]:
import os

os.environ['HTTP_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['HTTPS_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['no_proxy'] = "169.254.169.254,127.0.0.1"

In [2]:
%%sh

sudo su

cat <<EOF >> /etc/sysconfig/docker
export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1
EOF

service docker restart

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


## Build Docker images for train & serve

### Build generic train image : MXNET + Gluonts
*Do not execute cell, image already exists in ECR*

In [ ]:
%%sh

export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1

# Image name
image_name=mxnet-gluonts-training-0.3.3-cpu

cd container
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --registry-ids ${account} --no-include-email) #for docker you must authenticate both accounts, dkt and amazon
$(aws ecr get-login --region ${region} --registry-ids 763104351884 --no-include-email) #for amazon

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  --no-cache --pull --file 'Dockerfile_gluonts_training_0.3.3_cpu' -t ${image_name} .
docker tag ${image_name} ${fullname}
docker push ${fullname}

### Build training image ( from gluonts image + src code )

In [33]:
%%sh

export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1

# The name of our algorithm
algorithm_name=apo-demand-forecast-training

cd container
chmod +x train
#chmod -R +x src/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --registry-ids ${account} --no-include-email) #for docker you must authenticate both accounts, dkt and amazon

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  --no-cache --pull --file 'Dockerfile_train' -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}

Login Succeeded

Step 1/4 : FROM 150258775384.dkr.ecr.eu-west-1.amazonaws.com/mxnet-gluonts-training-0.3.3-cpu:latest
latest: Pulling from mxnet-gluonts-training-0.3.3-cpu
Digest: sha256:060009925c389f20c3be2b291cf1032097c7aaedff213456c789ed19a315056a
Status: Image is up to date for 150258775384.dkr.ecr.eu-west-1.amazonaws.com/mxnet-gluonts-training-0.3.3-cpu:latest
 ---> 31ee196156f7
Step 2/4 : COPY src/ /opt/program
 ---> 269d32b68776
Step 3/4 : COPY config/ /opt/program
 ---> f58cf8271300
Step 4/4 : ENTRYPOINT ["python"]
 ---> Running in d1966df114b1
Removing intermediate container d1966df114b1
 ---> efeab13f01c2
Successfully built efeab13f01c2
Successfully tagged apo-demand-forecast-training:latest
The push refers to repository [150258775384.dkr.ecr.eu-west-1.amazonaws.com/apo-demand-forecast-training]
3d6e4288adf5: Preparing
e2577ffe6788: Preparing
2b3fd2c0e44f: Preparing
98db11483f2b: Preparing
3190daa7d5b8: Preparing
8dfd6a9b4a23: Preparing
76f3ba8c633a: Preparing
6994e2dcf65c: 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [34]:
from sagemaker import get_execution_role

In [35]:
role = get_execution_role()
role

'arn:aws:iam::150258775384:role/FORECAST-SAGEMAKER-DEV'

In [36]:
bucket = 'fcst-workspace'
project_id = 'qlik/data/refined/domyos_nov_2019/gluonts_data' #/train_data_cutoff_201922.csv

## Bring your own Docker Image Sagemaker

In [37]:
image_name = '150258775384.dkr.ecr.eu-west-1.amazonaws.com/apo-demand-forecast-training:latest'

In [38]:
hyperparameters = {
        'yearly_order': 26,
        'quaterly_order': 5,
    
        'weekly_seasonality': False,
        'daily_seasonality': False,
        'yearly_seasonality': False,
    
        'n_changepoints': 36,
        'changepoint_range': 0.69,
        'changepoint_prior_scale': 1.91,
        'seasonality_prior_scale': 2.04
}

In [39]:
from sagemaker.estimator import Estimator
#from sagemaker.estimator import EstimatorBase

In [ ]:
instance_type = 'ml.m5.xlarge' #'local''ml.m5.2xlarge' #'local'#'ml.p3.2xlarge' # #'local' #'local_gpu'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=image_name,
                      hyperparameters=hyperparameters,
                      security_group_ids=["sg-0186b5ab868f43e42"],
                      subnets=["subnet-0f87a7ed73f4ead6d", "subnet-02c60aed04f0d4ee5"] #, entry_point='src/train.py'
                      )

estimator.fit('s3://'+bucket+'/'+project_id) #'/gluonts_ds_cutoff_201922.pkl' #job_name

## Bring your own hyperopt

In [11]:
import boto3

from time import gmtime, strftime
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [20]:
tuning_job_name = 'demand-forecast-prophet-tuning'# + strftime("%d-%H-%M-%S", gmtime())


hyperparameter_ranges = {
        'yearly_order': IntegerParameter (26, 29),#, #(1, 30)
        'quaterly_order': IntegerParameter(4, 6)#, #(1, 10)
}    
#        'n_changepoints': IntegerParameter(30, 32), #(1, 50)
#        'changepoint_range': ContinuousParameter(0.65, 0.69), #(0.6, 1.)
#        'changepoint_prior_scale': ContinuousParameter(1.8, 1.9, scaling_type="Logarithmic"), #1e-2, 1e2
#        'seasonality_prior_scale': ContinuousParameter(2.2, 2.4, scaling_type="Logarithmic")
#}


In [21]:
objective_metric_name = 'global_wape'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'global_wape',
                       'Regex': 'global_wape: ([0-9\\.]+)'}]

In [22]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

In [23]:
#tuner.fit({'training': 's3://'+bucket+'/'+project_id,
#          'test': 's3://'+bucket+'/'+project_id})

tuner.fit('s3://'+bucket+'/'+project_id)

In [24]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'InProgress'